In [2]:
import torch
import os
import numpy as np
import pandas as pd
import configparser

In [3]:
train_data_path = os.path.join("..","data","raw","MOT17","train")

In [4]:
train_folders =  sorted([folder_name 
                  for folder_name in os.listdir(train_data_path)
                  if "FRCNN" in folder_name])

In [5]:
sequence_lengths ={}
for folder_name in train_folders:
    config = configparser.ConfigParser()
    config.read(os.path.join(train_data_path,folder_name,"seqinfo.ini"))
    sequence_lengths[folder_name] = int(config["Sequence"]["seqLength"])
video_indices = np.argsort(train_folders)
sequence_lengths = torch.Tensor(list(zip(video_indices,list(sequence_lengths.values())))).to(torch.int64)

In [6]:
batch_size = 16
frames_size = 32

In [7]:
random_video_indices = torch.randint(len(sequence_lengths)-1,(batch_size,)) # get random video to extract frames
random_sequence_lengths = sequence_lengths[random_video_indices] - torch.tensor([0,frames_size]) # get total frames for videos and subtract frame size
randomizer = torch.vstack((torch.ones(batch_size),torch.rand(batch_size))).T # create randomizer for size of batch
random_start_timestamps = (randomizer*random_sequence_lengths).to(torch.int32) # generate random start time stamps

In [8]:
ground_truth_list = []
for i,folder in enumerate(train_folders):
    df = pd.read_csv(os.path.join(train_data_path, train_folders[i],"gt","gt.txt"),header=None)
    df.columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height", "x", "y", "z"]
    df["video_id"] = i
    df.drop(["x","y","z"],1,inplace=True)
    ground_truth_list.append(df)
ground_truth_df = pd.concat(ground_truth_list)  
ground_truth = torch.tensor(ground_truth_df.values)

In [9]:
detection_batch = []
for i,time_stamp in enumerate(random_start_timestamps):
    detections=ground_truth[torch.where((ground_truth[:,-1]==time_stamp[0]) & 
                               (ground_truth[:,0]>time_stamp[1]) & 
                               (ground_truth[:,0]< time_stamp[1]+frames_size))]
    detection_with_batch_idx = 
    torch.ones((detections.shape[0],detections.shape[1]+1))*i
    detection_with_batch_idx[:,:-1] = detections
    detection_batch.append(detection_with_batch_idx.to(torch.int32))

SyntaxError: invalid syntax (<ipython-input-9-036af73e54c4>, line 6)

In [353]:
random_video_index= torch.randint(len(sequence_lengths)-1,(1,)) # get random video to extract frames
random_sequence_lengths = sequence_lengths[random_video_indices] - torch.tensor([0,frames_size]) # get total frames for videos and subtract frame size
randomizer = torch.vstack((torch.ones(batch_size),torch.rand(batch_size))).T # create randomizer for size of batch
random_start_timestamps = (randomizer*random_sequence_lengths).to(torch.int32) # generate random start time stamps

In [354]:
sequence_lengths[random_video_index]

tensor([[  4, 654]])

In [17]:
random_video_index = torch.randint(len(sequence_lengths)-1,(1,)) # get random video to extract frames
sequence_length = sequence_lengths[random_video_index]
random_start = torch.Tensor([sequence_length[0][0],torch.randint(sequence_length[0][1],(1,))])

In [18]:
random_start

tensor([  2., 687.])

In [ ]:
for i,time_stamp in enumerate(random_start_timestamps):
    detections=ground_truth[torch.where((ground_truth[:,-1]==time_stamp[0]) & 
                               (ground_truth[:,0]>time_stamp[1]) & 
                               (ground_truth[:,0]< time_stamp[1]+frames_size))]
    detection_with_batch_idx = 
    torch.ones((detections.shape[0],detections.shape[1]+1))*i
    detection_with_batch_idx[:,:-1] = detections
    detection_batch.append(detection_with_batch_idx.to(torch.int32))

In [30]:
a=ground_truth[torch.where((ground_truth[:,-1]==random_start[0]) & 
                               (ground_truth[:,0]>=random_start[1]) & 
                               (ground_truth[:,0]< random_start[1]+frames_size))]

In [39]:
max_detections_in_frame = torch.unique(a[:,0],return_counts=True)[1].max()

In [127]:
unique_counts = torch.stack(torch.unique(a[:,0],return_counts=True)).T

In [129]:
unique_counts[:,1] = unique_counts[:,1].max()-unique_counts[:,1]

In [151]:
missing_detections = torch.repeat_interleave(unique_counts[:,0],unique_counts[:,1])

In [152]:
missing_detections_filler  =torch.zeros((missing_detections.shape[0],7))

In [153]:
missing_detections_filler[:,0] = missing_detections

In [154]:
missing_detections_filler[:,1] = -1

In [156]:
missing_detections_filler[:,-1] = 2

In [166]:
detections_from_frames = torch.vstack([a,missing_detections_filler])